In [1]:
import numpy as np

In [2]:
from tensorflow.keras.models import load_model

In [10]:
path_model = "ACAS_XU_tf_keras/ACASXU_1_1.h5"

In [11]:
model_11 = load_model(path_model)

In [16]:
model_35 = load_model("ACAS_XU_tf_keras/ACASXU_3_5.h5")

In [12]:
model_11.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_98 (Dense)             (None, 50)                300       
_________________________________________________________________
activation_98 (Activation)   (None, 50)                0         
_________________________________________________________________
dense_99 (Dense)             (None, 50)                2550      
_________________________________________________________________
activation_99 (Activation)   (None, 50)                0         
_________________________________________________________________
dense_100 (Dense)            (None, 50)                2550      
_________________________________________________________________
activation_100 (Activation)  (None, 50)                0         
_________________________________________________________________
dense_101 (Dense)            (None, 50)              

In [13]:
model_11.compile()

In [17]:
model_35.compile()

In [27]:
pt = np.array([1.0, 1.0, 1.0, 1.0, 1.0]).reshape(1,5)
model_11.predict(pt)

array([[0.19883643, 0.20087564, 0.19964513, 0.20093843, 0.19970441]],
      dtype=float32)

In [18]:
pt = np.array([1.0, 1.0, 1.0, 1.0, 1.0]).reshape(1,5)
model_35.predict(pt)

array([[0.1967246 , 0.2045978 , 0.19686562, 0.20457308, 0.19723897]],
      dtype=float32)

In [45]:
# import the necessary packages
from tensorflow.keras.losses import MSE
from tensorflow.keras.losses import categorical_crossentropy as CCE
import tensorflow as tf

In [58]:
def generate_adv_sample(model, x0, label, loss_function="MSE", eps=1e-5):
    # transforming into a tensorflow object
    x0_ = tf.cast(x0, tf.float32)
    
    # record our gradients
    with tf.GradientTape() as tape:
        # explicitly indicate that our input should be tacked for gradient updates
        tape.watch(x0_)

        # use our model to make predictions on the input and then compute the loss
        pred = model(x0_)
        if loss_function == "CCE":
            np_label = np.array([i==label for i in range(0,5)]).reshape((1,5))
            loss = CCE(np_label, pred)
        elif loss_function == "MSE":
            loss = MSE(label, pred)
        else:
            raise Exception("Unknown loss function '{0}'".format(loss_function))
        
        # calculate the gradients of loss with respect to the input, then compute the sign of the gradient
        gradient = tape.gradient(loss, x0_)
        signedGrad = tf.sign(gradient)

        # construct the image adversary
        adv_sample = (x0_ + (signedGrad * eps)).numpy()

        # return the adversarial sample to the calling function
        return(adv_sample)

In [62]:
x0_adv = generate_adv_sample(model_11, pt, label = 0, loss_function = "CCE", eps=1e-3)
print(x0_adv)

[[0.999 1.001 0.999 1.001 0.999]]


In [38]:
model_11.predict(x0_adv)

array([[0.19881836, 0.20089944, 0.19962737, 0.20096485, 0.19969004]],
      dtype=float32)

In [55]:
np_label = np.array([int(i==2) for i in range(0,5)]).reshape((1,5))
np_label

array([[0, 0, 1, 0, 0]])